### Testing Stopwords and Stemmers

In [1]:
text = "Esta unidade curricular de Processamento de Linguagem Natural é extremamente divertida e útil porque nos ajuda a desenvolver novas competências no campo da Aprendizagem Computacional não muito bem lecionada pelo Carlos Sexy"

Portuguese Stopwords

In [2]:
from nltk.corpus import stopwords

stopwords_list = stopwords.words('portuguese')

# Remover algumas palavras da lista, p.ex. "não"
stopwords_list.remove('não')

print(stopwords_list)

['de', 'a', 'o', 'que', 'e', 'é', 'do', 'da', 'em', 'um', 'para', 'com', 'uma', 'os', 'no', 'se', 'na', 'por', 'mais', 'as', 'dos', 'como', 'mas', 'ao', 'ele', 'das', 'à', 'seu', 'sua', 'ou', 'quando', 'muito', 'nos', 'já', 'eu', 'também', 'só', 'pelo', 'pela', 'até', 'isso', 'ela', 'entre', 'depois', 'sem', 'mesmo', 'aos', 'seus', 'quem', 'nas', 'me', 'esse', 'eles', 'você', 'essa', 'num', 'nem', 'suas', 'meu', 'às', 'minha', 'numa', 'pelos', 'elas', 'qual', 'nós', 'lhe', 'deles', 'essas', 'esses', 'pelas', 'este', 'dele', 'tu', 'te', 'vocês', 'vos', 'lhes', 'meus', 'minhas', 'teu', 'tua', 'teus', 'tuas', 'nosso', 'nossa', 'nossos', 'nossas', 'dela', 'delas', 'esta', 'estes', 'estas', 'aquele', 'aquela', 'aqueles', 'aquelas', 'isto', 'aquilo', 'estou', 'está', 'estamos', 'estão', 'estive', 'esteve', 'estivemos', 'estiveram', 'estava', 'estávamos', 'estavam', 'estivera', 'estivéramos', 'esteja', 'estejamos', 'estejam', 'estivesse', 'estivéssemos', 'estivessem', 'estiver', 'estivermos',

In [3]:
' '.join([w for w in text.split() if w not in stopwords_list])

'Esta unidade curricular Processamento Linguagem Natural extremamente divertida útil porque ajuda desenvolver novas competências campo Aprendizagem Computacional não bem lecionada Carlos Sexy'

Trying some Portuguese Stemmers

In [5]:
from nltk.stem import PorterStemmer, RSLPStemmer
from nltk.stem.snowball import PortugueseStemmer

ps = PorterStemmer()
snb = PortugueseStemmer()
rslp = RSLPStemmer()

text_list = text.split()

print("Porter Stemmer:\n", ' '.join([ps.stem(w) for w in text_list]))
print("\nSnowball Portugue Stemmer:\n", ' '.join([snb.stem(w) for w in text_list]))
print("\nRSLP Stemmer:\n",' '.join([rslp.stem(w) for w in text_list]))

Porter Stemmer:
 esta unidad curricular de processamento de linguagem natur é extremament divertida e útil porqu no ajuda a desenvolv nova competência no campo da aprendizagem computacion não muito bem lecionada pelo carlo sexi

Snowball Portugue Stemmer:
 esta unidad curricul de process de linguag natural é extrem divert e útil porqu nos ajud a desenvolv nov competent no camp da aprendizag computacional nã muit bem lecion pel carl sexy

RSLP Stemmer:
 est unidad curricul de process de lingu natur é extrem divert e útil porqu no ajud a desenvolv nov compet no camp da aprendiz computac não muit bem lecion pel carl sexy


### Dataset

In [6]:
import pandas as pd

dataset = pd.read_excel('OpArticles_ADUs.xlsx')

In [7]:
import re
from nltk.corpus import stopwords
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.model_selection import train_test_split
from sklearn.naive_bayes import ComplementNB
import sklearn.metrics as metrics
import time

stopwords_list = stopwords.words('portuguese')
stopwords_list.remove('não')

vectorizer = TfidfVectorizer(strip_accents='unicode')

y = dataset['label']

def test_stemmer(stemmer):
    corpus = []

    # Preprocess
    start = time.time()
    for i in range(0, dataset['tokens'].size):
        # get review, remove and lowercase non alpha chars
        review = re.sub('[^a-zA-Z\u00C0-\u00ff]', ' ', dataset['tokens'][i]).lower()
        # split into tokens, apply stemming and remove stop words
        review = ' '.join([stemmer.stem(w) for w in review.split() if w not in stopwords_list])
        corpus.append(review)
    stop = time.time()
    print("Preprocess time: %0.2fs" % (stop - start))

    # Fit vectorizer
    X = vectorizer.fit_transform(corpus).toarray()
    print("\n(Number of samples, Number of features):", X.shape)

    X_train, X_test, y_train, y_test = train_test_split(X, y, test_size = 0.20, random_state=0, stratify=y, shuffle=True)

    start = time.time()
    clf = ComplementNB()
    clf.fit(X_train, y_train)
    y_pred = clf.predict(X_test)
    stop = time.time()

    print("\nModel time: %0.2fs" % (stop - start))
    print("\nConfusion matrix:\n", metrics.confusion_matrix(y_test, y_pred))
    print("\nClassification report:\n", metrics.classification_report(y_test, y_pred))

### Porter

In [8]:
test_stemmer(ps)

Preprocess time: 2.73s

(Number of samples, Number of features): (16743, 15018)

Model time: 0.63s

Confusion matrix:
 [[306  21 279  45  82]
 [  3  55  61   8   6]
 [277  73 964  97 210]
 [ 41  22 108  92  19]
 [ 72  32 184  22 270]]

Classification report:
               precision    recall  f1-score   support

        Fact       0.44      0.42      0.43       733
      Policy       0.27      0.41      0.33       133
       Value       0.60      0.59      0.60      1621
    Value(+)       0.35      0.33      0.34       282
    Value(-)       0.46      0.47      0.46       580

    accuracy                           0.50      3349
   macro avg       0.42      0.44      0.43      3349
weighted avg       0.51      0.50      0.51      3349



### Portuguese Snowball

In [9]:
test_stemmer(snb)

Preprocess time: 3.16s

(Number of samples, Number of features): (16743, 8773)

Model time: 0.34s

Confusion matrix:
 [[300  30 292  40  71]
 [  8  41  68   8   8]
 [272  68 974 103 204]
 [ 42  12 113  94  21]
 [ 64  21 191  17 287]]

Classification report:
               precision    recall  f1-score   support

        Fact       0.44      0.41      0.42       733
      Policy       0.24      0.31      0.27       133
       Value       0.59      0.60      0.60      1621
    Value(+)       0.36      0.33      0.35       282
    Value(-)       0.49      0.49      0.49       580

    accuracy                           0.51      3349
   macro avg       0.42      0.43      0.43      3349
weighted avg       0.51      0.51      0.51      3349



### RSLP

In [10]:
test_stemmer(rslp)

Preprocess time: 5.25s

(Number of samples, Number of features): (16743, 7843)

Model time: 0.30s

Confusion matrix:
 [[298  23 292  46  74]
 [  8  38  70  10   7]
 [270  65 954 112 220]
 [ 38  12 117  95  20]
 [ 69  29 190  20 272]]

Classification report:
               precision    recall  f1-score   support

        Fact       0.44      0.41      0.42       733
      Policy       0.23      0.29      0.25       133
       Value       0.59      0.59      0.59      1621
    Value(+)       0.34      0.34      0.34       282
    Value(-)       0.46      0.47      0.46       580

    accuracy                           0.49      3349
   macro avg       0.41      0.42      0.41      3349
weighted avg       0.50      0.49      0.50      3349

